In [3]:
from pytorch_lightning import Trainer
from catinous.CatsinomModel import CatsinomModel
from catinous.CatsinomDataset import CatsinomDataset
import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import os
import torchvision.models as models
import torch.nn as nn
import torch
import torch.nn.functional as F
import argparse
import pytorch_lightning.logging as pllogging
import pytorch_lightning as pl

import numpy as np

%load_ext autoreload
%autoreload 2

### Dataset learning

In [9]:
dsname = 'lr'

def trained_full_dataset(dsname = 'lr', base_only=False, continue_from=None, continous=False, streamdirection='lr->hr', transition_phase_after=0.7):
    
    hparams = dict()
    hparams['root_dir'] = '/project/catinous/cat_data/'
    hparams['datasetfile'] = 'catsinom_' + dsname + '_dataset.csv'
    hparams['batch_size'] = 8
    hparams['train_iterations'] = 3000
    hparams['continous'] = continous
    hparams['streamdirection'] = streamdirection
    hparams

    if base_only:
        hparams['trainsplit'] = ['base_train']
        dsname = dsname + 'base'
    else:
        hparams['trainsplit'] = ['train','base_train']
    
    model = CatsinomModel(argparse.Namespace(**hparams))

    if continue_from is not None:
        model.load_state_dict(torch.load('/project/catinous/trained_models/' + continue_from + '_iterations.pt'))
        outname = '/project/catinous/trained_models/' + continue_from + dsname + '_iterations.pt'
        dsname = dsname + continue_from
    else:
        outname = '/project/catinous/trained_models/' + dsname + '_iterations.pt'
        
    if not os.path.exists(outname):
        logger = pllogging.TestTubeLogger('catinous_log_iterations', name=dsname)
        trainer = Trainer(gpus=1, max_epochs=1, early_stop_callback=False, logger=logger, val_check_interval=500)
        trainer.fit(model)
        torch.save(model.state_dict(), outname)

    return outname

In [8]:
trained_full_dataset('hr')
trained_full_dataset('lr')
trained_full_dataset('combined')
trained_full_dataset('hr', base_only=True)
trained_full_dataset('lr', base_only=True)
trained_full_dataset('hr', continue_from='lr')
trained_full_dataset('lr', continue_from='hr')

'/project/catinous/trained_models/hrlr_iterations.pt'

### Continous learning

In [6]:
trained_full_dataset('combined', continue_from='lrbase',
                    ,continous=True, streamdirection='lr->hr', transition_phase_after=0.7)

TypeError: trained_full_dataset() got an unexpected keyword argument 'streamdirection'

In [9]:
pllogging.__all__

['MLFlowLogger', 'TensorBoardLogger']

In [4]:
pllogging

<module 'pytorch_lightning.logging' from '/home/jhofmanninger/anaconda3/envs/candid/lib/python3.6/site-packages/pytorch_lightning/logging/__init__.py'>

In [29]:
np.random.seed(9375)
perm = np.random.permutation([1,2,3,4,5,6])
print(perm)

[4 2 1 3 5 6]
